In this notebook we are going to implement a first named entity model for the scrapped data from wikipedia

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('scrapped_wikidata.csv')


In [3]:
data

,espèce,description
0,Schefflera abyssinica,Le genre Schefflera constitue un groupe d’arbr...
1,Schefflera Mannii,Schefflera mannii (Hook.f.) Harms est une espè...
2,Tessmannia africana,Tessmannia est un genre de plantes dicotylédon...
3,Alternanthera sessilis,Alternanthera est un genre qui regroupe plus d...
4,Pimpinella Ledermannii,Pimpinella ledermannii H. Wolff est une espèce...
...,...,...
141,Monopetalanthus Hedinii,Aphanocalyx hedinii est une espèce d'arbres de...
142,Monopetalanthus microphyllus,Monopetalanthus heitzii est une espèce de plan...
143,Deinbollia dasybotrys,Deinbollia est un genre de plantes de la famil...
144,Hydrocotyle sibthorpioides,Les hydrocotyles (Hydrocotyle) forment un genr...


In [29]:
import re
TRAIN_DATA=[]
for idx,row in data.iterrows():
    entities=[]
    first_indexes=[m.start() for m in re.finditer(row['espèce'].lower(), row['description'].lower())]
    for fr in first_indexes:
        entities.append([fr,fr+len(row['espèce']),'espèce'])
    if entities:
        TRAIN_DATA.append([row['description'].lower(),{"entities":entities}])

In [30]:
len(TRAIN_DATA)

41

In [16]:
#trying to copy and run some code
import srsly
import typer
import warnings
from pathlib import Path

import spacy
from spacy.tokens import DocBin

def convert(lang: str, TRAIN_DATA, output_path: Path):
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in TRAIN_DATA:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is None:
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                warnings.warn(msg)
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(output_path)

In [31]:
convert("fr", TRAIN_DATA, "ner_data/train.spacy")
convert("fr", TRAIN_DATA, "ner_data/valid.spacy")

In [32]:
!python -m spacy init fill-config ner_data/base_config.cfg ner_data/config.cfg

✔ Auto-filled config with all values
✔ Saved config
ner_data\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-11-29 12:42:03.711697: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2022-11-29 12:42:03.713229: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2022-11-29 12:42:03.714583: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cufft64_10.dll'; dlerror: cufft64_10.dll not found
2022-11-29 12:42:03.715845: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'curand64_10.dll'; dlerror: curand64_10.dll not found
2022-11-29 12:42:03.717291: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cusolver64_11.dll'; dlerror: cusolver64_11.dll not found
2022-11-29 12:42:03.718809: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dy

In [33]:
!python -m spacy train ner_data/config.cfg --paths.train ner_data/train.spacy --paths.dev ner_data/valid.spacy --output models/output

ℹ Saving to output directory: models\output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     30.67    4.43    2.30   59.52    0.04
  9     200         11.51    989.80  100.00  100.00  100.00    1.00
 20     400          4.39      8.90  100.00  100.00  100.00    1.00
 32     600          0.00      0.00  100.00  100.00  100.00    1.00
 47     800          0.00      0.00  100.00  100.00  100.00    1.00
 64    1000          0.00      0.00  100.00  100.00  100.00    1.00
 84    1200          0.00      0.00  100.00  100.00  100.00    1.00
108    1400          0.00      0.00  100.00  100.00  100.00    1.00
135    1600          0.0

2022-11-29 12:42:27.287421: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2022-11-29 12:42:27.289007: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2022-11-29 12:42:27.290513: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cufft64_10.dll'; dlerror: cufft64_10.dll not found
2022-11-29 12:42:27.291958: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'curand64_10.dll'; dlerror: curand64_10.dll not found
2022-11-29 12:42:27.293385: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cusolver64_11.dll'; dlerror: cusolver64_11.dll not found
2022-11-29 12:42:27.294787: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dy

In [37]:
def f_remove_accents(old):
    """
    Removes common accent characters, lower form.
    Uses: regex.
    """
    new = old.lower()
    new = re.sub(r'[àáâãäå]', 'a', new)
    new = re.sub(r'[èéêë]', 'e', new)
    new = re.sub(r'[ìíîï]', 'i', new)
    new = re.sub(r'[òóôõö]', 'o', new)
    new = re.sub(r'[ùúûü]', 'u', new)
    return new

In [46]:
trained_nlp = spacy.load("models/output/model-best")
text = """Manilkara  lacera est une  espèce  typique  du  bord  de la 
mer et plus  précisément  du  front  de mer,  sur  les plages.  Il 
y constitue  parfois  des  fourrés  denses,  impénétrables,  taillés  en 
biseau  par  les embruns.  A l'abri  de ces  fourrés,  des  arbustes"""
text=text.replace("  "," ")
doc = trained_nlp(f_remove_accents(text))

for ent in doc.ents:
    print (ent.text, ent.label_)

manilkara lacera espèce
